# 1 Dependencies import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scripts.quickml.reader import read_csv
from scripts.quickml.plot import feature_distribution_univar, feature_filling, correlation_heatmap, feature_trend_bivar, feature_distribution_bivar, dataframe_distribution_overview, feature_distribution_bivar_box
from scripts.quickml.compare import index_lth
from scripts.quickml.conversion import feature_to_list
from scripts.utils import fill_infos

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

np.random.seed(294697)

***
# 2 Data import

In [ ]:
data = pd.read_csv("datas/sample.csv", index_col=[0], sep=";")

In [ ]:
data.shape

***
# 3 Overview

In [ ]:
dataframe_distribution_overview(data, figsize=(60, 6))

In [ ]:
data.info()

In [ ]:
data.describe()

***
# 4 Correlations

***
## 4.1 Identification

In [ ]:
categs = data.select_dtypes(include="object")
categs.info()

In [ ]:
categs.drop(columns=["code", "url", "quantity", "nutriscore_grade", "ecoscore_grade_fr", "image_url", "image_ingredients_url", "image_nutrition_url"], inplace=True)
categs.info()

***
## 4.2 Transformation

In [ ]:
def categorize(feature):
    new_feature = feature.astype("category").cat.codes
    return new_feature

In [ ]:
categs_to_nums = categs.apply(lambda col: categorize(col))
categs_to_nums

In [ ]:
for col in categs_to_nums.columns:
    categs_to_nums.rename(columns={col: col + "_CATEG"}, inplace=True)

categs_to_nums

In [ ]:
data = data.join([categs_to_nums])

In [ ]:
data.info()

***
## 4.3 Visualization

***
### 4.3.1 Original dataset

In [ ]:
correlation_heatmap(data)

***
### 4.3.2 Filtered dataset

In [ ]:
data = data.loc[:, data.columns.difference(data.loc[:, "energy-kcal_100g":"iron_100g"].columns)]

In [ ]:
data = data.loc[:, data.columns.difference(["created_t", "last_modified_t", "categories_en_CATEG", "creator_CATEG", "product_name_CATEG", "serving_quantity", "countries_en_CATEG", "labels_en_CATEG", "brand_owner_CATEG"])]

In [ ]:
correlation_heatmap(data)

***
# 5 Nutriscore / Ecoscore trends

In [ ]:
feature_trend_bivar(data, "nutriscore_score", "Nutriscore", "ecoscore_score_fr", "Ecoscore", 1000)

Nutriscore: the bigger the value, the worst the product is.  
Ecoscore: the bigger the value, the worst the environemental impact is.

## Conclusion:
The worst is a product for the health (high nutriscore), the better it is for the environement (low ecoscore).

***
# 6 Nutriscore / Ecoscore distribution

In [ ]:
feature_distribution_bivar(data, "nutriscore_score", "Nutriscore", "ecoscore_score_fr", "Ecoscore", divider=10)

***
# 7 Nutriscore / countries distribution

In [ ]:
temp = data[data.countries_en.notna() & data.countries_en.str.contains(",")].countries_en.head()
temp

In [ ]:
data = feature_to_list(data, "countries_en")

In [ ]:
data.loc[temp.index.values].countries_en

In [ ]:
data = data.explode("countries_en")

In [ ]:
data.loc[temp.index.values].countries_en

Let's see now how the nutriscore is distributed by country.

In [ ]:
data_by_countries_nutriscore = data.groupby(["countries_en", "nutriscore_grade"]).nutriscore_grade.agg([len]).reset_index()
data_by_countries_nutriscore.head()

In [ ]:
feature_distribution_univar(data_by_countries_nutriscore, "nutriscore_grade", "Nutriscore")

In [ ]:
nutriscore_distrib_per_country = data_by_countries_nutriscore.loc[data_by_countries_nutriscore.countries_en.isin(["France", "Germany", "Belgium", "Canada", "Italy", "Mexico", "Netherlands", "Poland", "Portugal", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States"])]

In [ ]:
plt.figure(figsize=(30,6))

sns.catplot(x="countries_en", y="len", hue="nutriscore_grade", data=nutriscore_distrib_per_country, kind="bar", height=5, aspect=5)

plt.title("Nutriscore grade distribution", size=20)
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel("Nutriscore grade", size=16)
plt.ylabel("Volume", size=16)
plt.show()

***
# 8 Ecoscore / countries distribution

Let's see now how the ecoscore is distributed by country.

In [ ]:
data_by_countries_ecoscore = data.groupby(["countries_en", "ecoscore_grade_fr"]).ecoscore_grade_fr.agg([len]).reset_index()
data_by_countries_ecoscore.head()

In [ ]:
feature_distribution_univar(data_by_countries_ecoscore, "ecoscore_grade_fr", "Ecoscore")

In [ ]:
ecoscore_distrib_per_country = data_by_countries_ecoscore.loc[data_by_countries_ecoscore.countries_en.isin(["France", "Germany", "Belgium", "Canad", "Italy", "Mexico", "Netherlands", "Poland", "Portugal", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States"])]

In [ ]:
plt.figure(figsize=(30,6))

sns.catplot(x="countries_en", y="len", hue="ecoscore_grade_fr", data=ecoscore_distrib_per_country, kind="bar", height=5, aspect=5)

plt.title("Nutriscore grade distribution", size=20)
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel("Nutriscore grade", size=16)
plt.ylabel("Volume", size=16)
plt.show()

***
# 9 Nutriscore / brands distribution

***
## 9.1 Data transformation

In [ ]:
data.brands.describe()

In [ ]:
data = feature_to_list(data, "brands").explode("brands")

In [ ]:
result = data.groupby(["brands"]).nutriscore_score.agg(["count", "min", "mean", "max", "std"]).dropna().sort_values(by="mean", ascending=False).reset_index()
result = result.loc[result["count"] >= 10]
result_concat = pd.concat([result.head(15), result.tail(15)])
result_concat

***
## 9.2 The 10 worst brands

In [ ]:
worst_5 = result.iloc[:10]
worst_5

In [ ]:
feature_distribution_bivar_box(data.loc[data.brands.isin(worst_5["brands"].values)], "brands", "nutriscore_score", (20, 10))

***
## 9.3 The 10 better brands

In [ ]:
better_5 = result.iloc[-10:]
better_5

In [ ]:
feature_distribution_bivar_box(data.loc[data.brands.isin(better_5["brands"].values)], "brands", "nutriscore_score", (20, 10))

***
# 10 Categories / brands distribution

In [ ]:
result_concat

In [ ]:
result_concat.brands.values

In [ ]:
data[data.brands == "Kinder"].loc[:, ["brands", "product_name", "main_category_en"]]

In [ ]:
brands_main_categ = data.groupby("brands").main_category_en.agg(freq = (lambda x: x.mode())).loc[result_concat.brands.values].explode("freq")
brands_main_categ

In [ ]:
brands_main_categ.describe()

In [ ]:
feature_distribution_univar(brands_main_categ, "freq", "Nutriscore")